## 纹理合成

参考论文：Graphcut Textures: Image and Video Synthesis Using Graph Cuts

In [412]:
import networkx as nx
import cv2
import numpy as np

In [ ]:
path = "./a.png" # 图片路径
a = 500 # 目标长度 (像素)
b = 500 # 目标宽度 (像素)

patch = np.array(cv2.imread(path))

In [414]:
res = np.zeros([a, b, 3], "uint8")
mask = np.zeros([a, b], "int32")
# mask 代表从第几张图来

a_p = patch.shape[0]
b_p = patch.shape[1]

### 简单拼接

In [415]:
count = 0
while False:
    count += 1

    # 寻找未放置下标
    index = np.where(mask == 0)
    if np.shape(index)[1] == 0:
        break

    # 抽取一个
    temp = np.random.randint(np.shape(index)[1])

    # 中间位置 x y
    x_m = index[0][temp]
    y_m = index[1][temp]

    # 左上角位置 x y
    if x_m < (a_p - 1) // 2: x = 0
    elif x_m > a - a_p // 2 - 1: x = a - a_p
    else: x = x_m - (a_p - 1) // 2
    if y_m < (b_p - 1) // 2: y = 0
    elif y_m > b - b_p // 2 - 1: y = b - b_p
    else: y = y_m - (b_p - 1) // 2

    res[x:x+a_p, y:y+b_p, :] = patch
    mask[x:x+a_p, y:y+b_p] = count

# cv2.imwrite("./res_paste.jpg", res)

### 纹理合成

In [416]:
res = np.zeros([a, b, 3], "uint8")
mask = np.zeros([a, b], "int32")

# coor[x, y] 当 mask 在该点非零时 指该点对应 patch 中的 x, y 坐标
coor = np.zeros([a, b, 2], "int32")

# s 与 t 指在新的 patch 中的坐标
# cor_s 指 第一张图 在 s 处对应 patch 中的坐标
# cor_t 指 第二张图 在 t 处对应 patch 中的坐标
def M(s, t, cor_s, cor_t):
   # patch 不会变 全局利用
   global patch

   # 若相邻点在某图中不存在 直接以最近的存在点替换
   dif = [t[0] - s[0], t[1] - s[1]]
   As = patch[cor_s[0], cor_s[1]]
   if cor_s[0] + dif[0] < 0 or cor_s[0] + dif[0] >= a_p \
         or cor_s[1] + dif[1] < 0 or cor_s[1] + dif[1] >= b_p:
      At = As
   else:
      At = patch[cor_s[0] + dif[0], cor_s[1] + dif[1]]
   Bt = patch[cor_t[0], cor_t[1]]
   if cor_t[0] - dif[0] < 0 or cor_t[0] - dif[0] >= a_p \
         or cor_t[1] - dif[1] < 0 or cor_t[1] - dif[1] >= b_p:
      Bs = Bt
   else:
      Bs = patch[cor_t[0] - dif[0], cor_t[1] - dif[1]]

   # 从 A, B 在 s, t 的值计算原始的差异 Mo
   Mo = np.sqrt(np.sum((As - Bs) ** 2)) + np.sqrt(np.sum((At - Bt) ** 2))

   # 进一步计算梯度 调用中只会出现 dif 为 [0, 1] 与 [1, 0]
   if dif[0] == 0:
      if cor_s[1] == 0:
         Asm = As
      else:
         Asm = patch[cor_s[0], cor_s[1] - 1]
      if cor_s[1] > b_p - 3:
         Atp = As
      else:
         Atp = patch[cor_s[0], cor_s[1] + 2]
      gas = np.sqrt(np.sum((At - Asm) ** 2))
      gat = np.sqrt(np.sum((Atp - As) ** 2))
      if cor_t[1] == b_p - 1:
         Btp = Bt
      else:
         Btp = patch[cor_t[0], cor_t[1] + 1]
      if cor_t[1] < 2:
         Bsm = Bs
      else:
         Bsm = patch[cor_t[0], cor_t[1] - 2]
      gbs = np.sqrt(np.sum((Bt - Bsm) ** 2))
      gbt = np.sqrt(np.sum((Btp - Bs) ** 2))
   else:
      if cor_s[0] == 0:
         Asm = As
      else:
         Asm = patch[cor_s[0] - 1, cor_s[1]]
      if cor_s[0] > a_p - 3:
         Atp = As
      else:
         Atp = patch[cor_s[0] + 2, cor_s[1]]
      gas = np.sqrt(np.sum((At - Asm) ** 2))
      gat = np.sqrt(np.sum((Atp - As) ** 2))
      if cor_t[0] == a_p - 1:
         Btp = Bt
      else:
         Btp = patch[cor_t[0] + 1, cor_t[1]]
      if cor_t[0] < 2:
         Bsm = Bs
      else:
         Bsm = patch[cor_t[0] - 2, cor_t[1]]
      gbs = np.sqrt(np.sum((Bt - Bsm) ** 2))
      gbt = np.sqrt(np.sum((Btp - Bs) ** 2))
   return Mo / (gas + gat + gbs + gbt)


In [417]:
count = 0
while True:
    count += 1

    # 寻找未放置下标
    index = np.where(mask == 0)
    if np.shape(index)[1] != 0:
        # 抽取一个
        temp = np.random.randint(np.shape(index)[1])

        # 中间位置 x y
        x_m = index[0][temp]
        y_m = index[1][temp]
    elif count == 20: break
    else:
        x_m = np.random.randint(a)
        y_m = np.random.randint(b)

    # 左上角位置 x y
    if x_m < (a_p - 1) // 2: x = 0
    elif x_m > a - a_p // 2 - 1: x = a - a_p
    else: x = x_m - (a_p - 1) // 2
    if y_m < (b_p - 1) // 2: y = 0
    elif y_m > b - b_p // 2 - 1: y = b - b_p
    else: y = y_m - (b_p - 1) // 2

    # 根据 mask 构造子图 但不连接切割线与新图:
    G = nx.Graph()
    node = 0
    c_v = np.zeros([a_p, b_p], "int32")

    ## 遍历 mask 顶点 根据非零顶点建立结点 维护一个 坐标偏移到顶点的映射
    for i in range(a_p):
        for j in range(b_p):
            if mask[i + x, j + y]:
                node += 1
                G.add_node(node, type = "point", x = i, y = j)
                c_v[i, j] = node

    ## 遍历纵边 通过 coor 确定 切割点 连接的权值
    for i in range(a_p):
        for j in range(b_p - 1):
            if mask[x + i, y + j] and mask[x + i, y + j + 1]:
                if mask[x + i, y + j] == mask[x + i, y + j + 1]:
                    G.add_edge(c_v[i, j], c_v[i, j + 1],
                               capacity = M([i, j], [i, j + 1], coor[x + i, y + j], [i, j + 1]))
                else:
                    node += 1
                    # 默认 竖直方向为 (i, j) 到 (i, j + 1)
                    G.add_node(node, type = "cut", x = i, y = j, dir = "v")
                    G.add_edge(node, c_v[i, j],
                               capacity = M([i, j], [i, j + 1], coor[x + i, y + j], [i, j + 1]))
                    G.add_edge(node, c_v[i, j + 1],
                               capacity = M([i, j], [i, j + 1], [i, j], coor[x + i, y + j + 1]))

    ## 遍历横边 通过 coor 确定 切割点 连接的权值
    for i in range(a_p - 1):
        for j in range(b_p):
            if mask[x + i, y + j] and mask[x + i + 1, y + j]:
                if mask[x + i, y + j] == mask[x + i + 1, y + j]:
                    G.add_edge(c_v[i, j], c_v[i + 1, j],
                               capacity = M([i, j], [i + 1, j], coor[x + i, y + j], [i + 1, j]))
                else:
                    node += 1
                    # 默认 水平方向为 (i, j) 到 (i + 1, j)
                    G.add_node(node, type = "cut", x = i, y = j, dir = "h")
                    G.add_edge(node, c_v[i, j],
                               capacity = M([i, j], [i + 1, j], coor[x + i, y + j], [i + 1, j]))
                    G.add_edge(node, c_v[i + 1, j],
                               capacity = M([i, j], [i + 1, j], [i, j], coor[x + i + 1, y + j]))
    
    # 遍历连通分支:
    for sub in nx.connected_components(G):
        subG : nx.Graph = nx.subgraph(G, sub).copy()

        # 添加一个虚拟源点 S
        subG.add_node(0, type="source")
        # 将在 patch 边缘的点连至 S
        for node in subG.nodes:
            if subG.nodes[node]["type"] == "point":
                t0 = subG.nodes[node]["x"]
                t1 = subG.nodes[node]["y"]
                if t0 == 0 or t0 == a_p - 1 or t1 == 0 or t1 == b_p - 1:
                    subG.add_edge(0, node, capacity = 1e10)

        # 添加一个虚拟汇点 T 与切割线连接 由 coor 得到权值
        subG.add_node(-1, type="sink")
        for node in subG.nodes:
            if subG.nodes[node]["type"] == "cut":
                t0 = subG.nodes[node]["x"]
                t1 = subG.nodes[node]["y"]
                if subG.nodes[node]["dir"] == "v":
                    subG.add_edge(-1, node,
                                capacity = M([t0, t1], [t0, t1 + 1],
                                            coor[x + t0, y + t1], coor[x + t0, y + t1 + 1]))
                else:
                    subG.add_edge(-1, node,
                                capacity = M([t0, t1], [t0 + 1, t1],
                                            coor[x + t0, y + t1], coor[x + t0 + 1, y + t1]))
         
        # 若无切割线 T 须与某点连接
        # 与 T 相连的点要求：不在 patch 边缘 但在连通分支边缘
        if subG.degree(-1) == 0:
            for node in subG.nodes:
                if subG.nodes[node]["type"] == "point":
                    t0 = subG.nodes[node]["x"]
                    t1 = subG.nodes[node]["y"]
                    if t0 > 0 and t0 < a_p - 1 and t1 > 0 and t1 < b_p - 1:
                        if not mask[t0, t1 - 1] * mask[t0, t1 + 1] * \
                                mask[t0 - 1, t1] * mask [t0 + 1, t1]:
                            subG.add_edge(-1, node, capacity = 1e10)

        # 若无这样的点 取中心点即可
        if subG.degree(-1) == 0:
            subG.add_edge(-1, c_v[a_p // 2, b_p // 2], capacity = 1e10)

        ## 通过源 S 汇 T 求解 最小割问题
        min_cut_res = nx.minimum_cut(subG, 0, -1)
        print(min_cut_res[0])
        print(min_cut_res[1][0].__len__())
        print(min_cut_res[1][1].__len__())
        new_patch = min_cut_res[1][1]
        #new_patch = nx.minimum_cut(subG, 0, -1)[1][1]

        ## 将与虚拟结点同一部分的点 赋予 新图的值 并对应改 mask 与 coor
        for node in new_patch:
            if subG.nodes[node]["type"] == "point":
                t0 = subG.nodes[node]["x"]
                t1 = subG.nodes[node]["y"]
                res[x + t0, y + t1] = patch[t0, t1]
                mask[x + t0, y + t1] = count
                coor[x + t0, y + t1] = [t0, t1]

    # 将无值的部分填充为新图 并对应改 mask 与 coor
    for i in range(a_p):
        for j in range(b_p):
            if not mask[i + x, j + y]:
                res[x + i, y + j] = patch[i, j]
                mask[x + i, y + j] = count
                coor[x + i, y + j] = [i, j]
    
    cv2.imwrite("./res" + str(count) + ".jpg", res)
    print("Iteration " + str(count) + "done.")

cv2.imwrite("./res.jpg", res)

1


C:\Windows\Temp\ipykernel_21580\3215118085.py:65: RuntimeWarning: invalid value encountered in double_scalars
  return Mo / (gas + gat + gbs + gbt)


89.06079528397363
700
3894
2


C:\Windows\Temp\ipykernel_21580\3215118085.py:65: RuntimeWarning: divide by zero encountered in double_scalars
  return Mo / (gas + gat + gbs + gbt)


10000000000.0
3865
1
3
7.090677838060134
15732
1
4
82.02082485386096
31462
7
5
47.06296796102463
28578
6
6
208.14889843328706
34649
2019
7
260.32345568136526
27133
16059
8
265.614410904831
49212
27
9
259.34685471517247
50210
45
10
309.34511413572386
19874
27157
11
224.52312895807142
51169
123
12
318.40435815923985
51061
245
13
348.90170152980176
51139
498
14
296.4413636679587
30433
20966
15
319.54243642820256
51470
228
16
289.5495345952759
41929
9750
17
247.73302899299813
36403
15138
18
402.94420455439064
37311
14203
19


True